https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a
https://www.browserstack.com/guide/find-element-by-xpath-in-selenium
https://stackoverflow.com/questions/3030487/is-there-a-way-to-get-the-xpath-in-google-chrome
https://www.guru99.com/selenium-webtable.html
https://stackoverflow.com/questions/19200497/python-selenium-webscraping-nosuchelementexception-not-recognized
https://www.w3schools.com/python/python_lists_add.asp


USe try and catch exceptions to stop iterating and go to the next page or line
https://www.scrapingbee.com/blog/selenium-python/

Selecting from a dropdown
https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python

waiting for page load
https://stackoverflow.com/questions/26566799/wait-until-page-is-loaded-with-selenium-webdriver-for-python
https://www.browserstack.com/guide/selenium-wait-for-page-to-load
https://www.askpython.com/python/examples/python-wait-for-a-specific-time

Possible CLI
https://github.com/markfink/whaaaaat

Adding a database
https://realpython.com/python-mysql/




1. run loop for the column until the end of the row where NoSuchEl exception is trigger
2. have the outer loop move to the next column
3. run loop for the column until the end of the row where NoSuchEl exception is trigger
4. continue until last column which which will tirgger another no such elem exception
5. have selenium click the next page (third outer loop)
6. repeat steps 1-4 until an error is triggered which stops all looping
7. clean up text and send to mySQL db


Header start and end xpaths, use this for the outer loop
Start: //*[@id="pagedResults"]/div[2]/table/thead/tr/th[1]
End: //*[@id="pagedResults"]/div[2]/table/thead/tr/th[17]

Row 1, col 1
//*[@id="pagedResults"]/div[2]/table/tbody/tr[1]/td[1]

Ro1 1, col 2
//*[@id="pagedResults"]/div[2]/table/tbody/tr[1]/td[2]

In [242]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#from selenium.common.exceptions import TimeoutException
import time
import pandas as pd

In [222]:
driver = webdriver.Chrome(r"C:/Users/Charlie/Documents/GitHub/chromedriver.exe")

In [200]:
url = 'https://results.svetiming.com/Santa-Cruz-Triathlon/events/2021/Santa-Cruz-Triathlon/results'
driver.get(url)

In [ ]:
def get_data_sve (url):
    


page_num_data = driver.find_element_by_xpath('//*[@id="pagedResults"]/div[1]/div[2]/div/div/span[2]') #look up how many pages
page_num_text = page_num_data.text
page_num_list = page_num_text.split(" ")
page_num = int(page_num_list[1])
print(page_num)

page_num = 2 #overide for debugging
olympic = dict.fromkeys(['Bib','Name', 'Team Name', 'City', 'Start Time', 'Swim', 'T1', 'Bike', 'T2', 'Run', 'Chip Elapsed', 'Division', 'Div Place', 'Age', 'Age Place', 'Gender', 'Gender Place'])
delay = 5

print(type(olympic["Bike"]))



for page in range(0,page_num):
    i = 1
    print(page)
    for key in olympic:
        rows = []
        for r in range(1,55):
            try:
                rows.append(driver.find_element_by_xpath('//*[@id="pagedResults"]/div[2]/table/tbody/tr[' + str(r) + ']/td[' + str(i) + ']').text)
                #print(rows)
            except  NoSuchElementException:
                #print('nse')
                break
        if olympic[key] is None:
            olympic[key]=rows
        else:
            olympic[key].append(rows)
        i=i+1
    #click to the next page of records
    print("im clicking")
    apply = driver.find_element_by_xpath('//*[@id="pagedResults"]/div[1]/div[2]/ul/li[3]/a')
    apply.click()
    print("im waiting")
    #driver.implicitly_wait(10)
    time.sleep(5)
    print("done waiting")

       

print(olympic)

In [224]:
#get the data for the sprint

catagory = Select(driver.find_element_by_id('SelectedDivisionName'))
catagory.select_by_visible_text('SPRINT-AGE GROUP')

apply = driver.find_element_by_xpath('//*[@id="searchEventResults"]/div/div/span/button')
apply.click()

In [ ]:
event_dict2 = dict.fromkeys(['Bib','Name', 'Team Name', 'City', 'Start Time', 'Swim', 'T1', 'Bike', 'T2', 'Run', 'Chip Elapsed', 'Division', 'Div Place', 'Age', 'Age Place', 'Gender', 'Gender Place'])


i = 1
for key in event_dict2:
    rows = []
    for r in range(1,55):
        try:
            rows.append(driver.find_element_by_xpath('//*[@id="pagedResults"]/div[2]/table/tbody/tr[' + str(r) + ']/td[' + str(i) + ']').text)
            #print(rows)
        except  NoSuchElementException:
            #print('nse')
            break
    event_dict2[key]=rows
    i=i+1





print(event_dict2)

In [ ]:
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="pagedResults"]/div[2]/table/thead/tr/th[1]')))
        print ("Page is ready!")
    except TimeoutException:
        print ("Loading took too much time!")


In [264]:
def get_data_sve (url, event):
    driver.get(url)  #pull up the page
    time.sleep(5)  #wait for the page to load

    #Select the data set
    catagory = Select(driver.find_element_by_id('SelectedDivisionName'))  #go to the drop down
    catagory.select_by_visible_text(event) #select from the dropdown
    apply = driver.find_element_by_xpath('//*[@id="searchEventResults"]/div/div/span/button')  #select the apply button
    apply.click()  #click the apply button
    time.sleep(2)  #wait for the page to load

    #determine how many pages for each even need to be scrolled through
    page_num_text = driver.find_element_by_xpath('//*[@id="pagedResults"]/div[1]/div[2]/div/div/span[2]').text  # select the element with the page numbers and get its text
    page_num_list = page_num_text.split(" ")  #split the string up
    page_num = int(page_num_list[1])  #get the number
    #print(page_num) #print it for validation

    page_num = 2  # overide for debugging
    
    data_vals = dict.fromkeys(
        ['Bib', 'Name', 'Team Name', 'City', 'Start Time', 'Swim', 'T1', 'Bike', 'T2', 'Run', 'Chip Elapsed',
         'Division', 'Div Place', 'Age', 'Age Place', 'Gender', 'Gender Place'])

    #pull the data off the page
    #need to update this so the dictionary gets populated cleanly
    #maybe have the inner loop add the values directly to the dictionary key
    for page in range(1, page_num+1):
        print("Pulling from page: " + str(page)) #let you know what page you are on
        i = 1   #counter for the columns
        for key in data_vals:
            rows = []
            for r in range(1, 55):
                try:    #stop the loop when you run out of data to pull
                    rows.append(driver.find_element_by_xpath(
                        '//*[@id="pagedResults"]/div[2]/table/tbody/tr[' + str(r) + ']/td[' + str(i) + ']').text)
                    # print(rows)
                except  NoSuchElementException:
                    # print('nse')
                    break
            if data_vals[key] is None:  #deal with the empty dict edge case
                data_vals[key] = rows
            else:
                data_vals[key].extend(rows)
            i = i + 1

        # click to the next page of records
        print("Going to next page")
        apply = driver.find_element_by_xpath('//*[@id="pagedResults"]/div[1]/div[2]/ul/li[3]/a')
        apply.click()
        #wait for results to load
        print("Waiting for page to load")
        time.sleep(5)
        print("Done waiting")

    return data_vals

In [265]:
sprint = get_data_sve('https://results.svetiming.com/Santa-Cruz-Triathlon/events/2021/Santa-Cruz-Triathlon/results', 'SPRINT-AGE GROUP')

Pulling from page: 1
Going to next page
Waiting for page to load
Done waiting
Pulling from page: 2
Going to next page
Waiting for page to load
Done waiting


In [239]:
print(sprint)

{'Bib': ['1051', '978', '924', '1187', '943', '1103', '1146', '1138', '1218', '995', '1123', '1198', '998', '1105', '985', '1015', '990', '1020', '982', '1189', '1182', '1264', '1259', '1132', '1039', '1256', '1010', '1184', '1141', '1243', '1282', '1006', '1163', '979', '1112', '905', '1030', '1061', '946', '913', '1183', '1261', '1159', '1221', '1179', '1048', '971', '1085', '804', '1137', ['1108', '1288', '1046', '1234', '1237', '1209', '916', '955', '1231', '1029', '1120', '1115', '1171', '1199', '989', '1160', '1043', '1239', '1155', '1145', '1127', '1135', '1255', '1212', '1162', '1079', '1228', '1005', '1292', '1126', '904', '1129', '1001', '1004', '949', '1172', '930', '1216', '1253', '918', '1143', '999', '1247', '1206', '1097', '1153', '957', '1263', '1269', '1240'], ['1147', '1040', '1094', '910', '1220', '953', '1041', '1136', '1278', '1045', '1232', '933', '1056', '1284', '972', '1277', '171', '1267', '974', '1140', '1207', '908', '1251', '925', '1008', '1050', '1265', '99

In [240]:
olympic = get_data_sve('https://results.svetiming.com/Santa-Cruz-Triathlon/events/2021/Santa-Cruz-Triathlon/results', 'OLYMPIC-AGE GROUP')

Pulling from page: 1
im clicking
im waiting
done waiting
Pulling from page: 2
im clicking
im waiting
done waiting
Pulling from page: 3
im clicking
im waiting
done waiting
Pulling from page: 4
im clicking
im waiting
done waiting
Pulling from page: 5
im clicking
im waiting
done waiting
Pulling from page: 6
im clicking
im waiting
done waiting
Pulling from page: 7
im clicking
im waiting
done waiting
Pulling from page: 8
im clicking
im waiting
done waiting
Pulling from page: 9
im clicking
im waiting
done waiting
Pulling from page: 10
im clicking
im waiting
done waiting
Pulling from page: 11
im clicking
im waiting
done waiting
Pulling from page: 12
im clicking
im waiting
done waiting
Pulling from page: 13
im clicking
im waiting
done waiting
Pulling from page: 14
im clicking
im waiting
done waiting


In [266]:
print(sprint['Bib'])

['1051', '978', '924', '1187', '943', '1103', '1146', '1138', '1218', '995', '1123', '1198', '998', '1105', '985', '1015', '990', '1020', '982', '1189', '1182', '1264', '1259', '1132', '1039', '1256', '1010', '1184', '1141', '1243', '1282', '1006', '1163', '979', '1112', '905', '1030', '1061', '946', '913', '1183', '1261', '1159', '1221', '1179', '1048', '971', '1085', '804', '1137', '1108', '1288', '1046', '1234', '1237', '1209', '916', '955', '1231', '1029', '1120', '1115', '1171', '1199', '989', '1160', '1043', '1239', '1155', '1145', '1127', '1135', '1255', '1212', '1162', '1079', '1228', '1005', '1292', '1126', '904', '1129', '1001', '1004', '949', '1172', '930', '1216', '1253', '918', '1143', '999', '1247', '1206', '1097', '1153', '957', '1263', '1269', '1240']


In [267]:
df = pd.DataFrame(sprint)

In [268]:
df.head()

,Bib,Name,Team Name,City,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,Division,Div Place,Age,Age Place,Gender,Gender Place
0,1051,MANUEL HUECKEL,TEAM WY55,SAN FRANCISCO,09:35:00,00:10:11,00:03:08,00:31:19,00:00:44,00:19:10,01:04:34,SPRINT-AGE GROUP,1,23,O/A,M,1
1,978,TYLER CURRAN,,WHITE PLAINS,09:35:00,00:09:28,00:02:59,00:31:42,00:00:52,00:20:14,01:05:16,SPRINT-AGE GROUP,2,24,O/A,M,2
2,924,BRIAN BERGER,GOLD COUNTRY TRIATHLON CLUB,LINCOLN,09:35:00,00:09:33,00:03:53,00:33:12,00:01:25,00:22:02,01:10:08,SPRINT-AGE GROUP,3,25,O/A,M,3
3,1187,CLAY ROBBINS,,REDWOOD CITY,09:35:00,00:11:30,00:03:42,00:34:03,00:01:37,00:19:26,01:10:19,SPRINT-AGE GROUP,4,30,1,M,4
4,943,ANDREW BUNNER,,BELMONT,09:40:00,00:11:46,00:03:49,00:33:34,00:01:06,00:20:54,01:11:12,SPRINT-AGE GROUP,5,44,1,M,5


In [269]:
df.tail()

,Bib,Name,Team Name,City,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,Division,Div Place,Age,Age Place,Gender,Gender Place
95,1153,JEFF OSBORNE,,SANTA CRUZ,09:40:00,00:11:52,00:05:24,00:49:20,00:01:14,00:24:08,01:32:00,SPRINT-AGE GROUP,96,52,11,M,74
96,957,MARIE CHAISSON,,SAN JOSE,09:45:00,00:15:50,00:05:13,00:41:18,00:01:42,00:28:10,01:32:15,SPRINT-AGE GROUP,97,45,4,F,23
97,1263,HANNAH VOELKER,,SAN MATEO,09:45:00,00:12:39,00:07:16,00:47:33,00:01:18,00:23:32,01:32:21,SPRINT-AGE GROUP,98,25,6,F,24
98,1269,MARK WATSON,SILICON VALLEY TRIATHLON CLUB,LOS GATOS,09:40:00,00:14:08,00:05:20,00:37:40,00:01:49,00:33:23,01:32:23,SPRINT-AGE GROUP,99,65,2,M,75
99,1240,KAAN TARIMAN,,LOS GATOS,09:40:00,00:14:39,00:06:25,00:41:13,00:01:47,00:28:31,01:32:38,SPRINT-AGE GROUP,100,42,8,M,76
